In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/cis/home/dpacker/my_documents/vote-counts


In [2]:
import pandas as pd
from pathlib import Path
import numpyro
from numpyro import distributions as dist
from numpyro.infer import NUTS, MCMC, Predictive
from jax import random, numpy as jnp
import matplotlib.pyplot as plt
import seaborn as sns
import os
from src import preprocessing

os.environ['CUDA_VISIBLE_DEVICES'] = '3'

path_data = Path("data")
path_raw_data = path_data / "raw_data"

In [3]:
raw_data = pd.read_csv(path_raw_data / "State_2020.csv")
# data = preprocessing.preprocess(raw_data)

/tmp/ipykernel_1555187/2800499606.py:1: DtypeWarning: Columns (0,11) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv(path_raw_data / "State_2020.csv")


In [9]:
group_cols = ["state", "district", "office", "candidate"]
vote_totals = raw_data[[*group_cols, "votes"]].groupby(group_cols).sum()

In [28]:
group_cols = ["county_fips", "state", "district", "office"]
raw_data[[*group_cols, "magnitude"]].groupby(group_cols).sum().index

MultiIndex([( 1001.0, 'ALABAMA',       '003', ...),
            ( 1001.0, 'ALABAMA',       '005', ...),
            ( 1001.0, 'ALABAMA',   'PLACE 1', ...),
            ( 1001.0, 'ALABAMA',   'PLACE 1', ...),
            ( 1001.0, 'ALABAMA',   'PLACE 1', ...),
            ( 1001.0, 'ALABAMA',   'PLACE 2', ...),
            ( 1001.0, 'ALABAMA',   'PLACE 2', ...),
            ( 1001.0, 'ALABAMA',   'PLACE 2', ...),
            ( 1001.0, 'ALABAMA', 'STATEWIDE', ...),
            ( 1003.0, 'ALABAMA',       '001', ...),
            ...
            (56043.0, 'WYOMING',       '005', ...),
            (56043.0, 'WYOMING',       '020', ...),
            (56043.0, 'WYOMING',       '027', ...),
            (56043.0, 'WYOMING', 'STATEWIDE', ...),
            (56043.0, 'WYOMING', 'STATEWIDE', ...),
            (56045.0, 'WYOMING',       '001', ...),
            (56045.0, 'WYOMING',       '002', ...),
            (56045.0, 'WYOMING',       '006', ...),
            (56045.0, 'WYOMING', 'STATEWIDE', ..

In [56]:
df = raw_data[[*group_cols, "votes"]].groupby(group_cols).sum()
df = df.reset_index(level=[1,2,3])
df["election_name"] = df["state"].astype(str) + "|" + df["district"].astype(str) + "|" + df["office"].astype(str)
# df["election_name"].to_dict()
county_to_election_name = {}

for key, row in df.iterrows():
  if key in county_to_election_name:
    county_to_election_name[key].append(row["election_name"])
  else:
    county_to_election_name[key] = [row["election_name"]]

In [57]:
county_to_election_name

{1001.0: ['ALABAMA|003|STATE BOARD OF EDUCATION',
  'ALABAMA|005|STATE BOARD OF EDUCATION',
  'ALABAMA|PLACE 1|COURT OF CIVIL APPEALS JUDGE',
  'ALABAMA|PLACE 1|COURT OF CRIMINAL APPEALS JUDGE',
  'ALABAMA|PLACE 1|SUPREME COURT ASSOCIATE JUSTICE',
  'ALABAMA|PLACE 2|COURT OF CIVIL APPEALS JUDGE',
  'ALABAMA|PLACE 2|COURT OF CRIMINAL APPEALS JUDGE',
  'ALABAMA|PLACE 2|SUPREME COURT ASSOCIATE JUSTICE',
  'ALABAMA|STATEWIDE|PUBLIC SERVICE COMMISSION PRESIDENT'],
 1003.0: ['ALABAMA|001|STATE BOARD OF EDUCATION',
  'ALABAMA|CIRCUIT 28, PLACE 5|CIRCUIT COURT JUDGE',
  'ALABAMA|PLACE 1|COURT OF CIVIL APPEALS JUDGE',
  'ALABAMA|PLACE 1|COURT OF CRIMINAL APPEALS JUDGE',
  'ALABAMA|PLACE 1|SUPREME COURT ASSOCIATE JUSTICE',
  'ALABAMA|PLACE 2|COURT OF CIVIL APPEALS JUDGE',
  'ALABAMA|PLACE 2|COURT OF CRIMINAL APPEALS JUDGE',
  'ALABAMA|PLACE 2|SUPREME COURT ASSOCIATE JUSTICE',
  'ALABAMA|STATEWIDE|PUBLIC SERVICE COMMISSION PRESIDENT'],
 1005.0: ['ALABAMA|PLACE 1|COURT OF CIVIL APPEALS JUDGE',
  '